In [ ]:
#@title ##### License { display-mode: "form" }
# Copyright 2019 DeepMind Technologies Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# OpenSpiel

* This Colab gets you started with installing OpenSpiel and its dependencies.
* OpenSpiel is a framework for reinforcement learning in games.
* The instructions are adapted from [here](https://github.com/deepmind/open_spiel/blob/master/docs/install.md).

## Install dependencies and clone repository

Let's first check the Python version, make sure to use a Python 3 runtime.

In [ ]:
!python --version

Clone [open_spiel](https://github.com/deepmind/open_spiel) repository and pull in source dependencies:  [pybind11](https://github.com/pybind/pybind11), [DDS](https://github.com/jblespiau/dds), [abseil](https://github.com/abseil)

In [ ]:
INSTALL_DIR = '/usr/local/open_spiel'

In [ ]:
!git config --global advice.detachedHead false
!git clone https://github.com/deepmind/open_spiel $INSTALL_DIR
!git clone -b 'v2.2.4' --single-branch --depth 1 https://github.com/pybind/pybind11.git $INSTALL_DIR/pybind11
!git clone -b 'develop' --single-branch --depth 1 https://github.com/jblespiau/dds.git  $INSTALL_DIR/open_spiel/games/bridge/double_dummy_solver
!git clone -b '20200225.1' --single-branch --depth 1 https://github.com/abseil/abseil-cpp.git $INSTALL_DIR/open_spiel/abseil-cpp

In [ ]:
#@title Optional dependencies: { display-mode: "both" }
BUILD_WITH_HANABI = False #@param {type:"boolean"}
BUILD_WITH_ACPC = False  #@param {type:"boolean"}
if BUILD_WITH_HANABI:
  %env BUILD_WITH_HANABI=ON
  !git clone -b 'master' --single-branch --depth 15 https://github.com/deepmind/hanabi-learning-environment.git $INSTALL_DIR/open_spiel/games/hanabi/hanabi-learning-environment
  !pushd $INSTALL_DIR/open_spiel/games/hanabi/hanabi-learning-environment && git checkout  'b31c973' && popd

if BUILD_WITH_ACPC:
  %env BUILD_WITH_ACPC=ON
  !git clone -b 'master' --single-branch --depth 1  https://github.com/jblespiau/project_acpc_server.git $INSTALL_DIR/open_spiel/games/universal_poker/acpc


Installing Python requirements:

In [ ]:
# we keep some baked-in Colab dependencies:
!sed  -e '/IPython/d' -e '/pip/d' -e '/matplotlib/d' $INSTALL_DIR/requirements.txt >> /tmp/requirements.txt
!pip3 install -r /tmp/requirements.txt

## Build `open_spiel`

In [ ]:
!apt-get install clang-9
!mkdir -p $INSTALL_DIR/build
%cd $INSTALL_DIR/build
!CXX=`which clang++-9` cmake -DPython_TARGET_VERSION=3.6 -DCMAKE_CXX_COMPILER=${CXX} ../open_spiel
!make -j$(nproc)
%cd /content

## Set `PYTHONPATH`

In [ ]:
import sys
import os
sys.path.append(INSTALL_DIR)
sys.path.append(os.path.join(INSTALL_DIR, 'build/python'))  # for pyspiel.so

In [ ]:
# verify that Python can find the open_spiel & pyspiel modules
import importlib
assert importlib.util.find_spec("open_spiel") is not None
assert importlib.util.find_spec("pyspiel") is not None

## (optional) Run `CMake` tests

In [ ]:
# run_python_test calls the python interpreter directly thus setting PYTHONPATH
%set_env PYTHONPATH=/env/python:$INSTALL_DIR:$INSTALL_DIR/build/python
!pushd $INSTALL_DIR/build && ctest -j$(nproc) --output-on-failure ../open_spiel && popd

# It's play time!

In [ ]:
import numpy as np
import pyspiel

game = pyspiel.load_game("tic_tac_toe")
state = game.new_initial_state()

while not state.is_terminal():
  state.apply_action(np.random.choice(state.legal_actions()))
  print(str(state) + '\n')